In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
url = '../dataset/'
df = pd.read_csv(url+'dataset_light.csv')

In [ ]:
df

# CB를 위한 product 정보들만 추출

In [7]:
cpy = df[['product_id','category_id','brand','first_level','secont_level','third_level','price']].drop_duplicates('product_id').copy()

In [8]:
cpy.set_index('product_id',inplace=True)
cpy.sort_index(inplace=True)
cpy

,category_id,brand,first_level,secont_level,third_level,price
product_id,,,,,,
1000978,2053013555631882655,samsung,electronics,smartphone,smartphone,0.00
1001588,2053013555631882655,meizu,electronics,smartphone,smartphone,128.31
1001606,2053013555631882655,apple,electronics,smartphone,smartphone,363.07
1002042,2053013555631882655,samsung,electronics,smartphone,smartphone,89.81
1002062,2053013555631882655,samsung,electronics,smartphone,smartphone,77.14
...,...,...,...,...,...,...
60500005,2162513070503494350,etc,construction,tools,welding,0.00
60500006,2162513070503494350,solaris,construction,tools,welding,29.83
60500007,2162513070503494350,denzel,construction,tools,welding,37.56


In [9]:
cpy['category_id'] = cpy['category_id'].apply(lambda x : str(x))
cpy.drop('category_id',axis=1,inplace=True)
# cpy.to_csv(url+'product_list.csv',) 

In [10]:
cpy

,brand,first_level,secont_level,third_level,price
product_id,,,,,
1000978,samsung,electronics,smartphone,smartphone,0.00
1001588,meizu,electronics,smartphone,smartphone,128.31
1001606,apple,electronics,smartphone,smartphone,363.07
1002042,samsung,electronics,smartphone,smartphone,89.81
1002062,samsung,electronics,smartphone,smartphone,77.14
...,...,...,...,...,...
60500005,etc,construction,tools,welding,0.00
60500006,solaris,construction,tools,welding,29.83
60500007,denzel,construction,tools,welding,37.56


In [11]:
def con_price(x):
    if x < 10:
        return str(int(round(x)))
    elif x < 100:
        return str(int(round(x,-1)))
    elif x < 1000:
        return str(int(round(x,-2)))
    elif x < 10000:
        return str(int(round(x,-3)))

cpy['price'] = cpy['price'].apply(lambda x : con_price(x))

## 데이터가 커 한번에 분류가 안되서 1분류로 1차적으로 나눈 뒤 진행

In [13]:
first_list = cpy['first_level'].drop_duplicates().to_list()
first_list

['electronics',
 'computers',
 'empty',
 'appliances',
 'construction',
 'auto',
 'furniture',
 'kids',
 'apparel',
 'sport',
 'stationery',
 'accessories',
 'medicine',
 'country_yard']

In [14]:
for category in tqdm(first_list):
    globals()['cpy_'+category] = cpy[cpy['first_level'] == category].copy()

100%|██████████| 14/14 [00:00<00:00, 93.55it/s]


In [15]:
for category in tqdm(first_list):
    globals()['info_'+category] = []
    for i in range(len(globals()['cpy_'+category])):
        (globals()['info_'+category]).append(' '.join((globals()['cpy_'+category]).iloc[i]))

100%|██████████| 14/14 [00:03<00:00,  3.91it/s]


In [ ]:
cpy_desktop

In [16]:
for category in tqdm(first_list):
    globals()['vect_'+category] = CountVectorizer()
    globals()['vect_'+category+'_tf'] = TfidfVectorizer()
    globals()[category] = (globals()['vect_'+category]).fit_transform(globals()['info_'+category])
    globals()[category+'_tf'] = (globals()['vect_'+category+'_tf']).fit_transform(globals()['info_'+category])
    globals()[category+'_df'] = pd.DataFrame((globals()[category]).toarray(), columns=sorted((globals()['vect_'+category]).vocabulary_))
    globals()[category+'_df_tf'] = pd.DataFrame((globals()[category+'_tf']).toarray(), columns=sorted((globals()['vect_'+category+'_tf']).vocabulary_))
    (globals()[category+'_df']).index = (globals()['cpy_'+category]).index
    (globals()[category+'_df_tf']).index = (globals()['cpy_'+category]).index

100%|██████████| 14/14 [00:01<00:00,  8.08it/s]


In [ ]:
first_list

In [17]:
for category in tqdm(first_list):
    globals()['cos_'+category] = cosine_similarity(globals()[category+'_df'],globals()[category+'_df'])
    # globals()['cos_'+category+'_tf'] = cosine_similarity(globals()[category+'_df_tf'],globals()[category+'_df_tf'])
    globals()['cos_'+category] = pd.DataFrame(globals()['cos_'+category], index=(globals()['cpy_'+category]).index, columns=(globals()['cpy_'+category]).index)
    # globals()['cos_'+category+'_tf'] = pd.DataFrame(globals()['cos_'+category+'_tf'], index=(globals()['cpy_'+category]).index, columns=(globals()['cpy_'+category]).index)

100%|██████████| 14/14 [15:57<00:00, 68.38s/it]


In [20]:
for category in tqdm(first_list):
    # globals()['cos_'+category] = cosine_similarity(globals()[category+'_df'],globals()[category+'_df'])
    # globals()['cos_'+category+'_tf'] = cosine_similarity(globals()[category+'_df_tf'],globals()[category+'_df_tf'])
    globals()['cos_'+category] = pd.DataFrame(globals()['cos_'+category], index=(globals()['cpy_'+category]).index, columns=(globals()['cpy_'+category]).index)

100%|██████████| 14/14 [00:00<00:00, 744.96it/s]


In [21]:
cos_accessories

product_id,18300000,18300003,18300004,18300005,18300006,18300010,18300011,18300012,18300015,18300016,...,52900189,52900190,52900191,52900192,52900193,52900194,52900195,52900196,52900197,52900198
product_id,,,,,,,,,,,,,,,,,,,,,
18300000,1.000000,0.771517,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.771517
18300003,0.771517,1.000000,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,...,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,0.771517,0.833333
18300004,0.714286,0.771517,1.000000,0.857143,0.857143,0.714286,0.714286,0.714286,0.714286,0.857143,...,0.714286,0.714286,0.857143,0.714286,0.857143,0.714286,0.714286,0.714286,0.714286,0.771517
18300005,0.714286,0.771517,0.857143,1.000000,1.000000,0.857143,0.714286,0.714286,0.857143,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.771517
18300006,0.714286,0.771517,0.857143,1.000000,1.000000,0.857143,0.714286,0.714286,0.857143,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.771517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52900194,0.714286,0.771517,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,1.000000,0.857143,0.714286,0.714286,0.771517
52900195,0.714286,0.771517,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,...,0.857143,0.714286,0.714286,0.857143,0.714286,0.857143,1.000000,0.714286,0.714286,0.771517
52900196,0.714286,0.771517,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,...,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,0.714286,1.000000,1.000000,0.771517


In [27]:
cos_accessories.loc[18300006].drop(18300006).sort_values()[-20:]

product_id
18300261    0.857143
18300693    0.857143
46900050    0.857143
18300259    0.857143
18300678    0.857143
21000028    0.857143
49800008    0.857143
18300248    0.857143
18300674    0.857143
18300260    0.857143
18300250    0.857143
18300667    0.857143
18300254    0.857143
49800000    0.857143
18300658    0.857143
18300656    0.857143
18300650    0.857143
18300252    0.857143
18300524    0.857143
18300005    1.000000
Name: 18300006, dtype: float64

countvector는 브랜드는 다르더라도 가격대가 비슷하고, tfidf는 브랜드가 더 우선적으로 유사도가 나온다.

In [30]:
cpy_accessories.loc[18300006]

brand               cameron
first_level     accessories
secont_level            bag
third_level             bag
price                    20
Name: 18300006, dtype: object

In [31]:
cpy_accessories.loc[18300005]

brand               cameron
first_level     accessories
secont_level            bag
third_level             bag
price                    20
Name: 18300005, dtype: object

In [32]:
cos_accessories.to_csv(url+'countvec/accessories.csv')

In [37]:
for category in tqdm(first_list):
    print(len(globals()['cos_'+category]),category)

100%|██████████| 14/14 [00:00<00:00, 18742.50it/s]

13886 electronics
7356 computers
80729 empty
11427 appliances
4771 construction
1141 auto
6382 furniture
4552 kids
11906 apparel
1879 sport
342 stationery
2315 accessories
29 medicine
157 country_yard


: 